Import libraries

In [1]:
#data loading and manipulation
import pandas as pd
import numpy as np

#for working with file paths
from os import path

#regular expressions
import re

#punctuation
import string

#spelling
#import enchant

In [2]:
pd.set_option('display.max_rows', 1000)

Load exempt data
* AQ06_where_data.csv is AQ06s submitted on or after 12/01/2016 and before 12/19/2016
* ANF_where_data.csv is ANFs submitted on or after 12/01/2016 and before 12/19/2016

In [3]:
aq06_path = path.join('data', 'AQ06_where_data.csv')
anf_path = path.join('data', 'ANF_where_data.csv')

aq06 = pd.read_csv(aq06_path)
anf = pd.read_csv(anf_path)

In [4]:
print aq06.shape
aq06.head()

(125, 7)


,ADDRESS,FORMTYPE,STICKER.,CONTRACTOR,STARTDATE,ENDDATE,EXPORTDATE
0,"ONE FEDERAL STREET, 25TH FLOOR, BOSTON, MA",AQ-06,100232092,LAKE CONTRACTING INC,11/16/2015 0:00:00,2/28/2016 0:00:00,12/19/2016 10:09:11
1,"500 COMMONWEALTH AVENUE, BOSTON, MA",AQ-06,100235507R1,J. L. DUNN & COMPANY,1/21/2016 0:00:00,7/31/2017 0:00:00,12/19/2016 10:09:11
2,"295 BURNETT ROAD, CHICOPEE, MA",AQ-06,100248580R1,"AMERICAN ENVIRONMENTAL, INC.",8/22/2016 0:00:00,5/30/2017 0:00:00,12/19/2016 10:09:11
3,"80 BROADWAY, EVERETT, MA",AQ-06,100249698R3,J.R. VINAGRO CORPORATION,1/9/2017 0:00:00,2/9/2017 0:00:00,12/19/2016 10:09:11
4,"0 MASSPIKE ROAD, BOSTON, MA",AQ-06,100251364R3,J.R. VINAGRO CORPORATION,10/28/2016 0:00:00,1/31/2017 0:00:00,12/19/2016 10:09:11


In [5]:
print anf.shape
anf.head()

(1165, 7)


,ADDRESS,FORMTYPE,STICKER.,CONTRACTOR,STARTDATE,ENDDATE,EXPORTDATE
0,"122 WAVERLY AVE, WATERTOWN, MA",ANF-001,100256143,NORTH SHORE ENVIRONMENTAL SERVICE,12/19/2016 0:00:00,12/19/2016 0:00:00,12/19/2016 10:08:48
1,"914 MAIN STREET, WORCESTER, MA",ANF-001,100256144,MORAN ENVIRONMENTAL RECOVERY LLC,12/7/2016 0:00:00,12/7/2016 0:00:00,12/19/2016 10:08:48
2,"1587 LONGMEADOW STREET, LONGMEADOW, MA",ANF-001,100255402R1,BAYSTATE CONTRACTING SERVICES INC,12/8/2016 0:00:00,12/8/2016 0:00:00,12/19/2016 10:08:48
3,"49 CHURCH STREET, NORTHBRIDGE, MA",ANF-001,100254916R2,NORTHEAST REMEDIATION,12/3/2016 0:00:00,12/7/2016 0:00:00,12/19/2016 10:08:48
4,"1000 WESTERN AVENUE, LYNN, MA",ANF-001,100256182,DEC-TAM CORPORATION,12/10/2016 0:00:00,12/10/2016 0:00:00,12/19/2016 10:08:48


Check for and reconcile missing data

In [6]:
aq06.isnull().sum()

ADDRESS       0
FORMTYPE      0
STICKER.      0
CONTRACTOR    0
STARTDATE     0
ENDDATE       0
EXPORTDATE    0
dtype: int64

In [7]:
aq06[aq06.FORMTYPE.isnull()]

,ADDRESS,FORMTYPE,STICKER.,CONTRACTOR,STARTDATE,ENDDATE,EXPORTDATE


In [8]:
def replace_null(df,col_name,value):
    df[col_name] = df[col_name].fillna(value)

replace_null(aq06,'FORMTYPE','AQ-06')

In [9]:
anf.isnull().sum()

ADDRESS       0
FORMTYPE      0
STICKER.      0
CONTRACTOR    0
STARTDATE     0
ENDDATE       0
EXPORTDATE    0
dtype: int64

In [10]:
open_today = pd.DataFrame()
open_today = open_today.append(aq06)
open_today = open_today.append(anf)
open_today.shape

(1290, 7)

In [11]:
#check for SAME
print open_today.loc[open_today['CONTRACTOR']=='SAME']

Empty DataFrame
Columns: [ADDRESS, FORMTYPE, STICKER., CONTRACTOR, STARTDATE, ENDDATE, EXPORTDATE]
Index: []


In [12]:
#check FORMTYPE values
print open_today['FORMTYPE'].value_counts()

ANF-001    1165
AQ-06       125
Name: FORMTYPE, dtype: int64


In [13]:
#clean up FORMTYPE
open_today.loc[open_today['FORMTYPE'].str.contains('06'),'FORMTYPE']='aq06'
open_today.loc[open_today['FORMTYPE'].str.contains('001'),'FORMTYPE']='anf001'

Remove punctuation from all text fields and convert to uppercase

In [14]:
#copy CONTRACTOR to CON to preserve original name; all edtis to be done on CON
for row in open_today:
    open_today['CON'] = open_today['CONTRACTOR']

In [15]:
#count unique CON
print len(open_today['CON'].value_counts())
open_today['CON'].value_counts()

193


DEC-TAM CORPORATION                          64
FJC ENTERPRISES INC                          59
AIR SAFE  INC                                51
NORTHSTAR CONTRACTING GROUP INC              43
ABIDE INC                                    39
NEW ROADS ENVIRONMENTAL SERVICES LLC         38
NATIONAL ABATEMENT INC                       36
NORTHEAST REMEDIATION                        31
BANNER ENVIRONMENTAL SERVICES INC            29
NON LICENSED REMOVAL                         25
WALSH ENVIRONMENTAL SERVICES                 22
AMERICAN ENVIRONMENTAL INC                   21
ENVIRONMENTAL RESPONSE SERVICES  INC         21
THE AULSON COMPANY INC                       20
ASBESTOS FREE  INC                           18
S & S ABATEMENT LLC                          18
BAYSTATE CONTRACTING SERVICES  INC           18
COMPASS RESTORATION SERVICE SERVICES  LLC    17
INFINITY ABATEMENT SERVICES INC              15
ACCOLADE ENVIRO                              15
CLEAN AIR ENVIRONMENTAL INC             

In [16]:
def upper_no_punct(df,col_name):
    for field in df[col_name]:
        new_field = field.translate(None, string.punctuation).upper()
        df[col_name] = df[col_name].replace(field,new_field)

upper_no_punct(open_today,'CON')
#370-->341 aq06 only
#502-->463 aq06 + anf

In [17]:
#Replaces 'pattern' in 'string' with 'sub' if 'pattern' starts 'string'
def lreplace(pattern, sub, string):
    return re.sub('^%s' % pattern, sub, string)

#Replaces 'pattern' in 'string' with 'sub' if 'pattern' ends 'string'
def rreplace(pattern, sub, string):
    return re.sub('%s$' % pattern, sub, string)

In [18]:
#standardize abbreviations and misspellings
def remove_abbrev(df, col_name, old_string, new_string):
    for field in df[col_name]:
        new_field = field.replace(old_string,new_string)
        df[col_name] = df[col_name].replace(field,new_field)

#remove abbreviations that appear at the end of the string only (ex: CO at the end, but not CO in CONSTRUCTION)
def remove_ending(df, col_name, ending):
    for field in df.loc[df[col_name].str.endswith(ending),col_name]:
        new_field = rreplace(ending,'',field)
        new_field = new_field.rstrip()
        df[col_name] = df[col_name].replace(field,new_field)

In [19]:
#slower?
abbrevs = {
    'AND':('ADN',),
    'BRO':('BROS',
           'BROTHER',
           'BROTHERS'),
    'BUILDING':('BUIDLING',
                'BUILIDNG'),
    'CAULFIELD':('CAULLFIELD',),
    'CONSTRUCTION':('CONSTRCTION',
                    'CONSTRCUTION',
                    'CONSTRUCION',
                    'CONSTRUCTON',
                    'CONSTRUCTUON',
                    'CONSTRUTION',
                    'CONTRUCTION',),
    'CORP':('CORPORATION',
            'CORPORATIO',
            'CORPOTATION',),
    'DESIGN':('DEISGN',),
    'DEMO':('DEMOLIITION',
            'DEMOLITION',
            'DEMOLTION'),
    'ENVIRONMENTAL':('ENVIRONMENATL',
                    'ENVIRONMETAL'),
    'INC':('INCORPORATED',),
    'JK SCANLAN':('JKS',),
    'SHAWMUT':('SAHWMUT',),
    'STRUCTURETONE':('STRUCTURE TONE',
                    'STURUCTURE TONE',),
    'SON':('SONS',),
    'DISMANTLING':('DISMANTLIING',),
}

spacing = {
    ' ':('  ',),
}

endings = [
    'LLC',
    'COINC',
    'INC',
    'COMPANY',
    'CORP',
    'LLP',
    'DESIGN',
    'CO',
    'CONC',
    'MANAGERS',
    'GROUP',
    'SERVICES',
    'SERVICE',
    'AND',
]

for new_string, old_strings in abbrevs.iteritems():
    for old_string in old_strings: 
        remove_abbrev(open_today, 'CON', old_string, new_string)

for ending in endings:
        remove_ending(open_today, 'CON', ending)
        
for new_string, old_strings in spacing.iteritems():
    for old_string in old_strings: 
        remove_abbrev(open_today, 'CON', old_string, new_string)
#341-->273 aq06 only
#463--382 aq06 + anf

In [20]:
#count unique CON
print len(open_today['CON'].value_counts())
open_today['CON'].value_counts()

173


DECTAM                                      66
FJC ENTERPRISES                             59
AIR SAFE                                    51
NORTHSTAR CONTRACTING                       44
ABIDE                                       39
NEW ROADS ENVIRONMENTAL                     38
NATIONAL ABATEMENT                          36
NORTHEAST REMEDIATION                       31
BANNER ENVIRONMENTAL                        29
THE AULSON                                  28
AMERICAN ENVIRONMENTAL                      26
NON LICENSED REMOVAL                        25
JR VINAGRO                                  23
WALSH ENVIRONMENTAL                         22
ENVIRONMENTAL RESPONSE                      21
ASBESTOS FREE                               18
BAYSTATE CONTRACTING                        18
S S ABATEMENT                               18
COMPASS RESTORATION                         17
ACCOLADE ENVIRO                             15
INFINITY ABATEMENT                          15
MORAN ENVIRON

#faster?
abbrevs = {
    'ADN':'AND',
    'BROS':'BRO',
    'BROTHER':'BRO',
    'BROTHERS':'BRO',
    'BUIDLING':'BUILDING',
    'BUILIDNG':'BUILDING',
    'CAULLFIELD':'CAULFIELD',
    'CONSTRCTION':'CONSTRUCTION',
    'CONSTRCUTION':'CONSTRUCTION',
    'CONSTRUCION':'CONSTRUCTION',
    'CONSTRUCTON':'CONSTRUCTION',
    'CONSTRUCTUON':'CONSTRUCTION',
    'CONSTRUTION':'CONSTRUCTION',
    'CONTRUCTION':'CONSTRUCTION',
    'CORPORATION':'CORP',
    'CORPORATIO':'CORP',
    'CORPOTATION':'CORP',
    'DEISGN':'DESIGN',
    'DEMOLIITION':'DEMO',
    'DEMOLITION':'DEMO',
    'DEMOLTION':'DEMO',
    'DISMANTLIING':'DISMANTLING',
    'ENVIRONMENATL':'ENVIRONMENTAL',
    'ENVIRONMETAL':'ENVIRONMENTAL',
    'INCORPORATED':'INC',
    'JKS':'JK SCANLAN',
    'SAHWMUT':'SHAWMUT',
    'STRUCTURE TONE':'STRUCTURETONE',
    'STURUCTURE TONE':'STRUCTURETONE',
    'SONS':'SON',    
}

spacing = {
    '  ':' ',
}

endings = [
    'LLC',
    'COINC',
    'INC',
    'COMPANY',
    'CORP',
    'LLP',
    'DESIGN',
    'CO',
    'CONC',
    'MANAGERS',
    'GROUP',
    'SERVICES',
    'SERVICE',
    'AND',
]

for old_string, new_string in abbrevs.iteritems():
    remove_abbrev(open_today, 'CON', old_string, new_string)

for ending in endings:
    remove_ending(open_today, 'CON', ending)
        
for old_string, new_strings in spacing.iteritems():
    remove_abbrev(open_today, 'CON', old_string, new_string)
#341-->273 aq06 only
#463--382 aq06 + anf

#standardize abbreviations and misspellings using using re.sub
def re_remove_abbrev(df, col_name, old_string, new_string):
    for field in df[col_name]:
        new_field = re.sub(old_string, new_string, old_string)
        df[col_name] = df[col_name].replace(field,new_field)

#fastest?
abbrevs = {
    'ADN':'AND',
    'BROS':'BRO',
    'BROTHER':'BRO',
    'BROTHERS':'BRO',
    'BUIDLING':'BUILDING',
    'BUILIDNG':'BUILDING',
    'CAULLFIELD':'CAULFIELD',
    'CONSTRCTION':'CONSTRUCTION',
    'CONSTRCUTION':'CONSTRUCTION',
    'CONSTRUCION':'CONSTRUCTION',
    'CONSTRUCTON':'CONSTRUCTION',
    'CONSTRUCTUON':'CONSTRUCTION',
    'CONSTRUTION':'CONSTRUCTION',
    'CONTRUCTION':'CONSTRUCTION',
    'CORPORATION':'CORP',
    'CORPORATIO':'CORP',
    'CORPOTATION':'CORP',
    'DEISGN':'DESIGN',
    'DEMOLIITION':'DEMO',
    'DEMOLITION':'DEMO',
    'DEMOLTION':'DEMO',
    'DISMANTLIING':'DISMANTLING',
    'ENVIRONMENATL':'ENVIRONMENTAL',
    'ENVIRONMETAL':'ENVIRONMENTAL',
    'INCORPORATED':'INC',
    'JKS':'JK SCANLAN',
    'SAHWMUT':'SHAWMUT',
    'STRUCTURE TONE':'STRUCTURETONE',
    'STURUCTURE TONE':'STRUCTURETONE',
    'SONS':'SON',    
}

spacing = {
    '  ':' ',
}

endings = [
    'LLC',
    'COINC',
    'INC',
    'COMPANY',
    'CORP',
    'LLP',
    'DESIGN',
    'CO',
    'CONC',
    'MANAGERS',
    'GROUP',
    'SERVICES',
    'SERVICE',
    'AND',
]

for old_string, new_string in abbrevs.iteritems():
    re_remove_abbrev(open_today, 'CON', old_string, new_string)

#for ending in endings:
#    remove_ending(open_today, 'CON', ending)
        
#for old_string, new_strings in spacing.iteritems():
#    remove_abbrev(open_today, 'CON', old_string, new_string)
#341-->273 aq06 only
#463--382 aq06 + anf

In [21]:
#write data to csv
open_today.to_csv('where.csv', columns=('ADDRESS','FORMTYPE','STICKER.','CON','STARTDATE','ENDDATE','EXPORTDATE'), header=False, index=False)

In [22]:
con = open_today[open_today['CON'].str.contains('NASDI')]
print con['FORMTYPE'].value_counts()

anf001    4
aq06      1
Name: FORMTYPE, dtype: int64
